In [1]:
# Import necessary libraries
import h2o
from h2o.automl import H2OAutoML

# Initialize the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyyhq4ud8
  JVM stdout: /tmp/tmpyyhq4ud8/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpyyhq4ud8/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,2 months and 1 day
H2O_cluster_name:,H2O_from_python_unknownUser_j643r1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
# Load your data into a H2OFrame
data = h2o.import_file('/kaggle/input/optiver-trading-at-the-close/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
# Specify the response and predictor columns
response_column = 'target'
predictor_columns = data.columns
predictor_columns.remove("target")

In [4]:
# Need to detect best model on reduced dataset due to size
## Determine the ratio for your sample
ratio = 0.01
## Split frame
splits = data.split_frame(ratios = [ratio], seed = 1234)
# Sample is the first split
sample = splits[0]
print("sample size train+val is:",sample.nrows)

# Split the sample into training and validation sets
train_sample, val_sample = sample.split_frame(ratios=[0.8], seed=1)

# Initialize and train the H2OAutoML model
#aml = H2OAutoML(max_models=20, seed=1)
aml = H2OAutoML(max_models=10,include_algos=["DeepLearning"])
aml.train(x=predictor_columns, y=response_column, training_frame=train_sample)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict on the validation set
preds = aml.predict(val_sample)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

sample size train+val is: 52388
AutoML progress: |
08:33:20.409: _train param, Dropping bad and constant columns: [row_id]

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                rmse      mse      mae    rmsle    mean_residual_deviance
DeepLearning_grid_2_AutoML_1_20231024_83320_model_1  9.38943  88.1614  6.37935      nan                   88.1614
DeepLearning_grid_3_AutoML_1_20231024_83320_model_2  9.39445  88.2557  6.38466      nan                   88.2557
DeepLearning_grid_3_AutoML_1_20231024_83320_model_3  9.40762  88.5032  6.3909       nan                   88.5032
DeepLearning_grid_2_AutoML_1_20231024_83320_model_3  9.41695  88.679   6.40378      nan                   88.679
DeepLearning_grid_1_AutoML_1_20231024_83320_model_1  9.41706  88.681   6.3996       nan                   88.681
DeepLearning_grid_1_AutoML_1_20231024_83320_model_2  9.42019  88.7401  6.39673      nan                   88.7401
Dee

In [5]:
# Extract best model
best_model = aml.leader

# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_grid_2_AutoML_1_20231024_83320_model_1'

In [6]:
# Split the data into training and validation sets
train, val = data.split_frame(ratios=[0.8], seed=1)

In [7]:
# Continue training the model on the full data set
best_model.train(x=predictor_columns, y=response_column, training_frame=train)

deeplearning Model Build progress: |

/opt/conda/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [row_id]
  warnings.warn(mesg["message"], RuntimeWarning)


█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1698136368506_5


Status of Neuron Layers: predicting target, regression, gaussian distribution, Quadratic loss, 43,601 weights/biases, 521.9 KB, 1,698,436 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  -----------------------
    1        15       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.040176483928070716   0.11082381010055542    0.0         0.0206859832839109     0.2150084376335144   0.2874694509142079     0.12888604402542114
    3        200      Rectifier  0.0        0.0   0.0   0.09432664787085086    0.10884451866149902    0.0         -0.03262675309288668   0.0795920193195343   0.9185978993839581     0.03071514517068863
    4        1        Linear                0.0   0.0   0.0008373354167815706  0.0005511087365448475  0.0         -0.002961692911576392  0.04871302843093872  0.0032737819804407433  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 86.78267366617993
RMSE: 9.315721854273018
MAE: 6.2662695523639735
RMSLE: NaN
Mean Residual Deviance: 86.78267366617993

Scoring History: 
    timestamp            duration          training_speed    epochs     iterations    samples      training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  ---------  ------------  -----------  ---------------  -------------------  --------------  -------------
    2023-10-24 09:04:01  0.000 sec                           0          0             0            nan              nan                  nan             nan
    2023-10-24 09:04:07  6.618 sec         9220 obs/sec      0.0115575  1             48435        9.44413          89.1916              6.33469         0.00481612
    2023-10-24 09:04:13  13.513 sec        12243 obs/sec     0.0346912  3             145384       9.39143          88.1989              6.3096          0.0158925
    2023-10-24 09:04:20  20.030 sec        13397 obs/sec     0.0578365  5             242381       9.36663          87.7337              6.29207         0.0210835
    2023-10-24 09:04:27  26.682 sec        13873 obs/sec     0.0809261  7             339145       9.35967          87.6033              6.29533         0.0225379
    2023-10-24 09:04:33  32.623 sec        14497 obs/sec     0.104073   9             436147       9.36008          87.6111              6.279           0.0224513
    2023-10-24 09:04:39  38.640 sec        14884 obs/sec     0.127238   11            533229       9.35267          87.4725              6.28428         0.0239982
    2023-10-24 09:04:45  44.617 sec        15177 obs/sec     0.150417   13            630367       9.38501          88.0783              6.30345         0.0172381
    2023-10-24 09:04:50  50.275 sec        15505 obs/sec     0.173597   15            727511       9.32463          86.9488              6.2662          0.0298411
    2023-10-24 09:04:56  56.187 sec        15688 obs/sec     0.196753   17            824554       9.32112          86.8832              6.26609         0.0305729
    2023-10-24 09:05:02  1 min  2.346 sec  15768 obs/sec     0.219886   19            921500       9.35807          87.5734              6.30071         0.0228717
    2023-10-24 09:05:08  1 min  7.884 sec  15990 obs/sec     0.243063   21            1.01863e+06  9.31786          86.8224              6.26138         0.0312509
    2023-10-24 09:05:13  1 min 13.403 sec  16186 obs/sec     0.266243   23            1.11577e+06  9.32235          86.9063              6.27386         0.0303153
    2

In [8]:
# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_model_python_1698136368506_5'

In [9]:
# Predict on the validation set
preds = best_model.predict(val)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
